Import libraries

In [1]:
import streamlit as st
from rdkit import Chem
import deepchem as dc
from rdkit.Chem.Draw import MolToImage
import joblib
import pandas as pd

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Load models

In [2]:
model_path_fluorescence = "C:/Users/pooja.d/Downloads/best_models (1)/best_models (1)/best_models/best_classifier.joblib"
model_fluorescence = joblib.load(model_path_fluorescence)

C:\Users\pooja.d\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\pooja.d\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
model_path_regression = "C:/Users/pooja.d/Downloads/best_models (1)/best_models (1)/best_models/new_best_regressor.joblib"
model_regression = joblib.load(model_path_regression)

Load emission max model

In [7]:
model_path_emission = "C:/Users/pooja.d/Downloads/best_models (1)/best_models (1)/best_models/best_regressor.joblib"
model_emission = joblib.load(model_path_emission)

Calculate Morgan fingerprints from SMILES string

In [11]:
def smiles_to_morgan(smiles):
    mol = Chem.MolFromSmiles(smiles) 
    featurizer = dc.feat.CircularFingerprint(radius=3, size=1024)
    fp = featurizer.featurize([mol])
    return fp[0]  # Take the features for the first (and only) sample

In [12]:
def smiles_to_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    featurizer = dc.feat.MACCSKeysFingerprint()
    descriptors = featurizer.featurize([mol])
    return pd.DataFrame(data=descriptors)

Predict absorption max

In [13]:
def predict_absorption_max(model, smiles, solvent):
    smiles_desc = smiles_to_descriptors(smiles)
    solvent_desc = smiles_to_descriptors(solvent)
    X = pd.concat([smiles_desc, solvent_desc], axis=1)
    y_pred = model.predict(X)
    absorption_max = y_pred[0]
    return absorption_max

In [14]:
def predict_emission_max(model, smiles, solvent):
    try:
        # Generate descriptors for the given SMILES string
        smiles_desc = smiles_to_descriptors(smiles)
        solvent_desc = smiles_to_descriptors(solvent)
        
        # Concatenate descriptors for SMILES and solvent
        X = pd.concat([smiles_desc, solvent_desc], axis=1)
        
        # Predict the emission max using the model
        emission_max_pred = model.predict(X)
        
        # Extract the predicted emission max value
        emission_max = emission_max_pred[0]  # Assuming it's a single value
        
        return emission_max
    
    except Exception as e:
        print(f"Error in predicting emission max: {e}")
        return None

Predict some value using a model

In [15]:
def predict(model, fp):
    try:
        print(f"Input shape: {fp.shape}")

        # Check if the model is a deepchem model
        if hasattr(model, 'predict'):
            # If it's a deepchem model, assume it's a classification model
            pred = model.predict([fp])[0]
        else:
            # If not a deepchem model, assume it's a traditional sklearn model
            pred = model.predict(fp)[0]
        print(f"Prediction: {pred}")
        return pred
    except Exception as e:
        print(f"Error in predict: {e}")
        return None

Draw structure

In [16]:
def draw_molecule(smiles):
    mol = Chem.MolFromSmiles(smiles)
    img = MolToImage(mol)
    return img
# Title
st.title("Model Selector")

2024-05-13 09:31:06.800 
  command:

    streamlit run C:\Users\pooja.d\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

Sidebar for selecting the model

In [12]:
model_selector = st.sidebar.selectbox("Select Model", ["Classification Model", "Absorption Max Model", "Emission Max Model"])

In [13]:
if model_selector == "Emission Max Model":
    st.title("Emission Max Prediction")

    # Input for emission max prediction
    smiles_emission_input = st.text_input("Enter a SMILES string for the molecule:")
    solvent_input = st.text_input("Enter a SMILES string for the solvent:")

    # Check if the input is valid and not empty
    if smiles_emission_input and solvent_input:
        try:
            # Predict using the emission max model
            emission_result = predict_emission_max(model_emission, smiles_emission_input, solvent_input)

            # Draw molecule structure from SMILES string
            st.image(MolToImage(Chem.MolFromSmiles(smiles_emission_input)), caption="Molecule Structure", width=100, use_column_width=True)

            # Display the predicted emission max value on the app
            st.write(f"Predicted Emission Max: {emission_result}")
            
        except Exception as e:
            # Display an error message if the input is invalid or cannot be processed
            st.error(f"Error in Emission Max Model: {e}")

In [17]:
elif model_selector == "Classification Model":
    st.title("Fluorescence Classifier")

    # Input for fluorescence model
    smiles_input = st.text_input("Enter a SMILES string:")

    # Check if the input is valid and not empty
    if smiles_input:
        try:
            # Calculate Morgan fingerprints from SMILES string
            fp = smiles_to_morgan(smiles_input)

            # Predict using the fluorescence model
            result = predict(model_fluorescence, fp)

            # Draw molecule structure from SMILES string
            image = draw_molecule(smiles_input)

            # Display the molecule structure and the prediction result on the app
            st.image(image, caption="Molecule Structure", width=100, use_column_width=True)
            
            # Check the result and display the prediction
            if result is not None:
                st.write(f"Prediction: {'Fluorescent Molecule' if result == 1 else 'Non-fluorescent Molecule'}")
        except Exception as e:
            # Display an error message if the input is invalid or cannot be processed 
            st.error(f"Error in Fluorescence Model: {e}")

SyntaxError: invalid syntax (2781572529.py, line 1)

In [16]:
else:
    st.title("Absorption Max Model")
    # Input for regression model
    smiles_input = st.text_input("Enter a SMILES string for the molecule:")
    solvent_input = st.text_input("Enter a SMILES string for the solvent:")
    if smiles_input and solvent_input:
        try:
            # Predict using the regression model
            result = predict_absorption_max(model_regression, smiles_input, solvent_input)

            # Draw molecule structure from SMILES string
            st.image(MolToImage(Chem.MolFromSmiles(smiles_input)), caption="Molecule Structure", width=100, use_column_width=True)

            # Display the prediction result on the app
            st.write(f"Predicted Absorption Max: {round(result, 2)}")
        except Exception as e:
            # Display an error message if the input is invalid or cannot be processed 
            st.error(f"Error in Regression Model: {e}")

SyntaxError: invalid syntax (1308479693.py, line 1)